In [ ]:
import os
import glob

import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 100   # Hm, not ideal. Shorten comments?
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

from strax import *
chio = io_chunked

# ADC->PE conversions for XENON1T
to_pe = 1e-3 * np.array([7.05, 0.0, 0.0, 8.09, 4.38, 7.87, 3.58, 7.5, 7.44, 4.82, 7.07, 5.79,  0.0, 5.55, 7.95, 7.02, 6.39, 8.1, 7.15, 7.43, 7.15, 11.4, 3.97, 7.28,  5.41, 7.4, 0.0, 0.0, 7.04, 7.27, 4.22, 16.79, 4.14, 7.04, 0.0, 5.38,  7.39, 7.02, 4.53, 5.17, 7.13, 5.48, 4.6, 7.33, 6.14, 6.52, 7.59,  4.76, 7.56, 7.54, 4.57, 4.6, 7.12, 8.0, 4.7, 8.68, 3.74, 4.97, 10.36,  7.53, 6.02, 12.45, 0.0, 4.49, 4.82, 0.0, 8.13, 7.27, 3.55, 5.65,  4.55, 8.64, 7.97, 0.0, 3.57, 3.69, 5.87, 5.12, 9.8, 0.0, 5.08, 4.09,  3.87, 8.17, 6.73, 9.03, 0.0, 6.93, 0.0, 6.52, 7.39, 0.0, 4.92, 7.48,  5.82, 4.05, 3.9, 5.77, 8.14, 7.62, 7.61, 5.55, 0.0, 7.12, 5.02, 4.57,  4.46, 7.44, 3.57, 7.58, 7.16, 7.33, 7.69, 6.03, 5.87, 9.64, 4.68,  7.88, 0.0, 10.84, 7.0, 3.62, 7.5, 7.45, 7.69, 7.69, 3.49, 3.61, 7.44,  6.38, 0.0, 5.1, 3.72, 5.22, 0.0, 0.0, 4.43, 0.0, 3.87, 0.0, 3.6,  5.35, 8.4, 5.1, 6.45, 5.07, 4.28, 3.5, 0.0, 7.28, 0.0, 4.25, 0.0,  4.72, 6.26, 7.28, 5.34, 7.55, 3.85, 5.54, 7.5, 7.31, 0.0, 7.76, 7.57,  6.66, 7.29, 0.0, 7.59, 3.8, 3.58, 5.21, 4.29, 7.36, 7.76, 4.0, 6.23,  5.86, 0.0, 7.34, 3.58, 3.57, 5.26, 0.0, 7.67, 4.05, 4.3, 4.21, 7.59,  7.59, 0.0, 6.41, 4.86, 3.73, 5.09, 7.59, 7.64, 7.7, 0.0, 5.25, 8.0,  5.32, 7.91, 0.0, 4.41, 11.82, 0.0, 4.51, 7.05, 8.63, 5.12, 4.45,  4.03, 0.0, 0.0, 3.54, 4.18, 9.5, 3.64, 3.67, 7.28, 3.59, 5.03, 3.6,  5.4, 7.18, 3.73, 6.21, 6.47, 3.7, 7.69, 4.58, 7.46, 6.74, 0.0, 3.66,  7.49, 7.55, 3.64, 0.0, 7.34, 4.06, 3.74, 3.97, 0.0, 4.29, 4.96, 3.77,  8.57, 8.57, 8.57, 8.57, 8.57, 8.57, 214.29, 171.43, 171.43, 171.43,  171.43, 171.43])
data_dir = os.path.abspath('./180219_2005')

In [ ]:
# This PMT is mad and oscillates terribly
to_pe[87] = 0

In [ ]:
# # Add first_time to records chunk metadata. Should have been done when writing the files...
# for fn in tqdm(chio.chunk_files(data_dir + '/records'),
#                desc='Adding first_time info'):
#     d, md = strax.load(fn, with_meta=True)
#     md['first_time'] = int(d['time'][0])   # Without int, json complains
#     strax.save_metadata(fn, **md)

In [ ]:
data_info('records')

# Build peaks

In [ ]:
@register_plugin
class ReducedRecords(StraxPlugin):
    data_kind = 'records'
    compressor = 'zstd'
    dtype = record_dtype()
    
    seen_bytes = 0
    
    def compute(self, records):
        r = records
        self.seen_bytes += r.nbytes
        integrate(r)
        r = exclude_tails(r, to_pe)
        return r
    
pl = ReducedRecords()
pl.save(data_dir)

In [ ]:
@register_plugin
class Peaks(StraxPlugin):
    data_kind = 'peaks'
    dtype = peak_dtype(n_channels=len(to_pe))
    
    def compute(self, reduced_records):
        r = reduced_records
        
        hits = find_hits(r)
        # strax.cut_outside_hits(r, hits)    # Was already done on conversion

        peaks = find_peaks(hits, to_pe, 
                                 result_dtype=self.dtype)
        sum_waveform(peaks, r, to_pe)
        
        peaks = split_peaks(peaks, r, to_pe)
        
        compute_widths(peaks)
        return peaks
    
Peaks().save(data_dir)

In [ ]:
data_info('peaks')

# Reduce peak info

In [ ]:
@register_plugin
class PeakBasics(StraxPlugin):
    dtype = [
        (('Peak integral in PE',
            'area'), np.float32),
        (('Number of PMTs contributing to the peak',
            'n_channels'), np.int16),
        (('PMT number which contributes the most PE',
            'max_pmt'), np.int16),
        (('Start time of the peak (ns since unix epoch)',
            'time'), np.int64),
        (('End time of the peak (ns since unix epoch)',
            'endtime'), np.int64),
        (('Width (in ns) of the central 50% area of the peak',
            'range_50p_area'), np.float32),
        (('Fraction of area seen by the top array',
            'area_fraction_top'), np.float32),
    ]

    def compute(self, peaks):
        p = peaks
        r = np.zeros(len(p), self.dtype)
        r['area'] = p['area']
        r['n_channels'] = (p['area_per_channel'] > 0).sum(axis=1)
        r['range_50p_area'] = p['width'][:,5]
        r['max_pmt'] = np.argmax(p['area_per_channel'], axis=1)
        r['time'] = p['time']
        r['endtime'] = p['time'] + p['dt'] * p['length']

        # TODO: get n_top_pmts from some config...
        area_top = (p['area_per_channel'][:,:127] 
                    * to_pe[:127].reshape(1, -1)
                   ).sum(axis=1)
        r['area_fraction_top'] = area_top/p['area']
        return r

PeakBasics().save(data_dir)

In [ ]:
data_info('peak_basics')

In [ ]:
df = chio.slurp_df(data_dir + '/peak_basics')
df.head()

This takes about 30x less memory than the raw peaks (with waveforms, area_per_channel, etc). A substantial reduction, but not enough to forego chunking.

In [ ]:
from multihist import Histdd
d = df
mh = Histdd(d['area'], d['range_50p_area'],
            bins=(np.logspace(0, 7, 100),
                  np.logspace(1, 4, 100)))
mh.plot(log_scale=True)
plt.xscale('log')
plt.yscale('log')

In [ ]:
d = df[df['n_channels'] > 3]
plt.scatter(d['area'], 
            d['range_50p_area'],
            c=d['area_fraction_top'], 
            s=0.1,
            cmap=plt.cm.rainbow, vmin=0, vmax=1)
plt.xscale('log')

plt.colorbar(label='Area fraction top')
plt.xlabel("Area (pe)")
plt.ylabel("Width (50% area, ns)")
plt.gca().patch.set_facecolor('black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1, 5e6)
plt.ylim(10, 2e4)

# Classification

In [ ]:
@register_plugin
class PeakClassification(StraxPlugin):
    dtype = [
        (('Classification of the peak.', 
            'type'), np.int8)
    ]
    
    def compute(self, peak_basics):
        p = peak_basics
        r = np.zeros(len(p), dtype=self.dtype)
        
        is_s1 = p['area'] > 100
        is_s1 &= p['range_50p_area'] < 150
        r['type'][is_s1] = 1
        
        is_s2 = p['area'] > 1e4
        is_s2 &= p['range_50p_area'] > 200
        r['type'][is_s2] = 2
        
        return r

PeakClassification().save(data_dir)

# Merging

In [ ]:
@register_plugin
class PeakInfo(MergePlugin):
    depends_on = ('peak_basics', 'peak_classification')
    

In [ ]:
df = PeakInfo().process_and_slurp(data_dir)

In [ ]:
data_info('peak_info')

# Event building

In [ ]:
@register_plugin
class Events(StraxPlugin):
    data_kind = 'events'
    dtype = [
        (('Event number in this dataset',
              'event_number'), np.int64),
        (('Event start time in ns since the unix epoch',
              'time'), np.int64),
        (('Event end time in ns since the unix epoch',
              'endtime'), np.int64),
    ]
    
    # Uh oh, state... must force sequential when we start doing multiprocessing
    events_seen = 0
    
    def compute(self, peak_basics):
        left_ext = int(1e6)
        right_ext = int(1e6)
        large_peaks = peak_basics[peak_basics['area'] > 1e5]
        
        # TODO: this can be done much faster
        event_ranges = []
        split_indices = np.where(np.diff(large_peaks['time']) > left_ext + right_ext)[0] + 1
        for ps in np.split(large_peaks, split_indices):
            start = ps[0]['time'] - left_ext
            stop = ps[-1]['time'] + right_ext
            event_ranges.append((start, stop))
        event_ranges = np.array(event_ranges)
        self.events_seen += len(event_ranges)

        result = np.zeros(len(event_ranges), self.dtype)
        result['time'], result['endtime'] = event_ranges.T
        result['event_number'] = np.arange(len(event_ranges)) + self.events_seen
        return result


In [ ]:
Events().save(data_dir)
events = Events().process_and_slurp(data_dir)

# Events do not overlap
assert np.min(events['time'][1:] - events['endtime'][:-1]) > 0

In [ ]:
data_info('events')

In [ ]:
@register_plugin
class EventBasics(LoopPlugin):
    depends_on = ('events', 'peak_basics', 'peak_classification')
    dtype = [(('Number of peaks in the event',
                   'n_peaks'), np.int32),
             
             (('Main S1 peak index',
                   's1_index'), np.int32),
             (('Main S1 area (PE)',
                   's2_area'), np.int32),
             (('Main S1 area fraction top',
                   's1_area_fraction_top'), np.float32),
             (('Main S1 width (ns, 50% area)',
                   's1_range_50p_area'), np.float32),
             
             (('Main S2 peak index',
                   's2_index'), np.int32),
             (('Main S2 area (PE)',
                   's1_area'), np.int32),
             (('Main S2 area fraction top',
                   's2_area_fraction_top'), np.float32),
             (('Main S2 width (ns, 50% area)',
                   's2_range_50p_area'), np.float32),
             
             (('Drift time between main S1 and S2 in ns',
                   'drift_time'), np.int64),
            ]
    
    def compute_loop(self, event, peaks):
        result = dict(n_peaks=len(peaks))
        if not len(peaks):
            return result
        
        main_s = dict()
        for s_i in [1, 2]:
            ss = peaks[peaks['type'] == s_i]
            if not len(ss):
                continue
            main_i = result[f's{s_i}_index'] = np.argmax(ss['area'])
            s = main_s[s_i] = ss[main_i]
            for prop in 'area area_fraction_top range_50p_area'.split():
                result[f's{s_i}_{prop}'] = s[prop]
                
        if len(main_s) == 2:
            result['drift_time'] = main_s[2]['time'] - main_s[1]['time']

        return result

In [ ]:
data_info('event_basics')

In [ ]:
ev_props = EventBasics().process_and_slurp(data_dir, n_per_iter=10)
df = pd.DataFrame.from_records(ev_props)

In [ ]:
plt.scatter(df['drift_time'] / int(1e3),
            df['s2_range_50p_area'] / int(1e3),
            c=df['s1_area_fraction_top'],
            vmin=0, vmax=0.25, cmap=plt.cm.jet,
            marker='.', edgecolors='none')
plt.colorbar(label="S1 area fraction top", extend='max')
plt.xlabel('Drift time (us)')
plt.ylabel('S2 width (us)')
plt.ylim(0, 4)
plt.tight_layout()

In [ ]:
data_info('peaks')

In [ ]:
@register_plugin
class LargestPeakArea(LoopPlugin):
    depends_on = ('events', 'peak_basics')
    dtype = [(('Area of largest peak in event (PE)',
                   'largest_area'), np.float32)]
    
    def compute_loop(self, event, peaks):
        x = 0
        if len(peaks):
            x = peaks['area'].max()

        return dict(largest_area=x)
    

In [ ]:
pd.DataFrame.from_records(LargestPeakArea().process_and_slurp(data_dir))

In [ ]:
# # Show we've been shown all the correct peaks
# ps = chio.slurp(data_dir + '/peak_basics')
# n_contained_in = np.bincount(fully_contained_in(ps, events) + 1)[1:]
# assert np.all(ev_props['n_peaks'] == n_contained_in)

# Find stuff to investigate (old, but useful functions also below)

In [ ]:
df = provider('peak_basics').process_and_slurp(data_dir)

In [ ]:
mask = df['n_channels'] >= 5
#mask &= ~np.in1d(max_pmt, [31, 87])
d = df[mask]

plt.scatter(d['area'], 
            d['range_50p_area'],
            c=d['area_fraction_top'], 
            s=0.1,
            cmap=plt.cm.rainbow, vmin=0, vmax=1)

plt.colorbar(label='Area fraction top')
plt.xlabel("Area (pe)")
plt.ylabel("Width (50% area, ns)")
plt.gca().patch.set_facecolor('black')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1, 5e6)
plt.ylim(10, 2e4)

# Waveform inspection tools

In [ ]:
def chunk_i(t, subdir='records'):
    chunk_starts = get_chunk_starts(subdir)
    i = np.searchsorted(chunk_starts, t) - 1
    if i < 0:
        # TODO: handle starting exactly at the last chunk
        raise ValueError("time before last chunk starts")
    # TODO: Assumes last chunk is infinitely long...
    return i
    
def get_data(t_start, t_end, channels=None, subdir='records'):
    """Return all things from subdir that overlap with [t_start, t_end]
    in channels.
    
    This is quite slow if you have big chunks.
    """
    chunk_start = chunk_i(t_start, subdir)
    chunk_end = chunk_i(t_end, subdir)
    in_files = chunk_files(subdir)
    result = []
    for i in range(chunk_start, chunk_end + 1):
        d = strax.load(in_files[i])
        d = d[(t_start < d['time'] + d['length'] * d['dt']) 
              & (d['time'] < t_end)]
        if channels is not None:
            d = d[np.in1d(d['channel'], channels)]
        result.append(d)
    return np.concatenate(result)
    
def plot_wvs(r, t0=None, time_unit='ns', alternate_colors=False, **kwargs):
    time_unit_str = time_unit
    time_unit_num = int(dict(ns=1, us=1e3, ms=1e6, s=1e9)[time_unit])

    t0 = r['time'][0]
    for i, d in enumerate(r):
        length = d['length']
        w = d['data'][:length]
        t = (np.arange(length, dtype=np.int64) * d['dt'] + (d['time'] - t0)) 
        if alternate_colors:
            color = 'k' if i % 2 == 0 else 'darkslategrey'
        else:
            color = 'k'
        plt.plot(t/time_unit_num, w/d['dt'], color=color, **kwargs)
        
    plt.xlabel("Time (%s)" % time_unit_str)
    plt.ylabel("Amplitude (pe/ns)")

# Try to view PMT waveforms

In [ ]:
df = strax.io_chunked.slurp_df(data_dir + '/peak_basics')

In [ ]:
sd = df[
    (df['area'] > 1e4)
    & (df['area_fraction_top'] > 0.9)
    & (df['max_pmt'] == 87)
]

In [ ]:
#get_data(d.time - before, d.endtime + after, subdir='peaks')

In [ ]:
def get_wv(t_start, t_end, subdir='peaks', channels=None, **kwargs):
    r = get_data(t_start, t_end, subdir=subdir, channels=channels)
    if len(r):
        plot_wvs(r, **kwargs)
    else:
        print("Nothing found")
    
def get_wv_of(x, extend=0, **kwargs):
    try:
        t_end = x['endtime']
    except KeyError:
        t_end = x['time'] + x['dt'] * x['length']
    get_wv(x['time'] - extend, t_end + extend,
            **kwargs)

In [ ]:
get_wv_of(sd.iloc[1], extend=int(1e5), 
          channels=[87], subdir='records',
          time_unit='us', alternate_colors=True)

In [ ]:
ts = get_chunk_starts('records')
detector_time = (ts[-1] - ts[0] + np.diff(ts).mean()) / int(1e9)

In [ ]:
!du -h {input_dir}/records

In [ ]:
# weirdo_is = np.where((peaks['area'] > 1e5) & (aft > 0.9))[0]

In [ ]:
def plot_peak(p, t0=None, **kwargs):
    n = p['length']
    if t0 is None:
        t0 = p['time']
    plt.plot((p['time'] - t0) + np.arange(n) * p['dt'], 
             p['data'][:n] / p['dt'], 
             linestyle='steps-mid',
             **kwargs)
    plt.xlabel("Time (ns)")
    plt.ylabel("Sum waveform (PE / ns)")
    
def plot_peaks(peaks):
    t0 = peaks[0]['time']
    for p in peaks:
        plot_peak(p, t0=t0,
                  label='%.1e PE, %d ns dt' % (p['area'], p['dt'], ))
    plt.ylim(0, None)

i = weirdo_is[0]
plot_peaks(peaks[i-1:i+5])
plt.legend(loc='best')
#plt.yscale('symlog')
plt.show()
aft[i-1:i+3]

In [ ]:
#peaks[max_pmt[]]